## Macro 2: Problem Sheet 1

### Exercise 1

> This notebook is structured as follows. First I define all necessary functions in a sort of abstract fashion. Then at the very end I call all functions to produce numerical results and plots.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from statsmodels.formula.api import ols

import seaborn as sns
import matplotlib.pyplot as plt

### Constants

In [2]:
DATA_PATH = Path("data/CNEF_PSID")
TEN_PERCENT_OF_ANNUAL_FULLTIME_HOURS = 208

### Functions

In [3]:
def get_yearly_variables(year):
    """Returns list and dict of relevant columns given year.
    
    Convert variables names to year specific names. For example instead
    of 'x11102' writes 'x1110285' for the year '85. Additionally add
    variable name for individual id.
    
    For details on variable names and more information seek the
    codebook: https://www.cnefdata.org/documentation/codebooks
    
    
    Args:
        year (int): Year. (Write 85 for 1985.)
        
    Returns:
        as_list (list): List of variable names.
        as_dict (dict): Dict of variable name translation to human
            readable form.
    
    """
    variables = {
        # "x11101ll": "individual",
        "x11102": "household",
        "i11102": "income",
        "d11105": "relationship_to_head",
        "d11101": "age",
        "d11106": "num_persons",
        "d11107": "num_children",
        "d11109": "education",
        "e11101": "hours",
        "e11102": "work",
    }
    as_dict = {f"{key}{year}": value for key, value in variables.items()}
    as_dict = dict(as_dict, **{"x11101ll": "individual"})
    as_list = [f"{key}{year}" for key in variables.keys()]
    as_list.append("x11101ll")
    return as_dict, as_list

In [4]:
def load_data_given_year(year):
    """Load data and assign new columns given year.
    
    This already does some steps described in part 2 of exercise 1.
    
    Args:
        year (int): Year. (Write 85 for 1985.)
        
    Returns:
        df (pd.DataFrame): Data frame with columss
    
    """
    cols_mapper, cols = get_yearly_variables(year)

    df = pd.read_stata(DATA_PATH / f"pequiv{year}.dta", columns=cols)

    df = df.rename(columns=cols_mapper)
    df = df.set_index(["household", "individual"])
    df = df.dropna(how="all")
    df = df.assign(
        **{
            "year": year,
            "num_adults": df.num_persons - df.num_children,
            "relationship_to_head": df.relationship_to_head.str.split(" ").apply(
                lambda s: s[0]
            ),
            "work": df.work.str[-1],
        }
    )
    df = df.drop(columns=["num_persons", "num_children"])
    df = df.convert_dtypes()
    return df

In [5]:
def subset_data(df):
    """Subset data frame.
    
    This does most steps described in part 2 of exercise 1.
    
    Args:
        df (pd.DataFrame): Frame produced by :func:`load_data_given_year`.
        
    Returns:
        df (pd.DataFrame): Cleaned data frame.
    
    """
    df = df.query("relationship_to_head in ['head', 'partner']")
    df = df.query("num_adults == 2")

    total = df.groupby(by="household")[["hours", "income"]].sum()

    df = df.query("relationship_to_head == 'head'")
    df = df.reset_index(level="individual", drop=True)
    df = df.assign(**{"income": total.income, "hours": total.hours})

    df = df.query("25 <= age < 56")
    df = df.query("hours >= @TEN_PERCENT_OF_ANNUAL_FULLTIME_HOURS")
    df = df.query("income > 0")

    df = df.assign(**{"income": np.log(df.income)})

    df = df.drop(["relationship_to_head", "num_adults", "hours"], axis=1)
    df = df.astype(
        {
            "income": float,
            "education": "category",
            "age": "category",
            "year": "category",
            "work": "category",
        }
    )
    df = df.set_index("age", append=True)
    df = df.dropna(how="any")
    return df

In [6]:
def load_and_clean_data():
    """Load, subset and merge data for years 1980 to 1997.
    
    Returns:
        df (pd.DataFrame): Cleaned and merged data frame with index ['household', 'year']
            and columns 'income', 'age', 'education' and 'work'. Column 'income' is float
            while all other columns are category.
    
    """
    dfs = []
    for year in range(80, 98):
        df = load_data_given_year(year)
        df = subset_data(df)
        dfs.append(df)

    df = pd.concat(dfs).sort_index()
    return df

In [7]:
def fit_dummy_regression(df):
    """Fit dummy regression on data in df.
    
    In the formula object C() tells statsmodels to use the variable as
    categorical variable.
    
    """
    df = df.reset_index()
    model = ols("income ~ C(year) + C(age) + C(education) + C(work)", data=df)
    model = model.fit()
    return model

In [8]:
def add_residuals_to_df(df, model):
    """Add column residuals to data frame."""
    df = df.reset_index()
    df = df.assign(**{"residuals": model.resid})
    df = df.set_index(["household", "age"])
    return df

### Computation

In [9]:
df = load_and_clean_data()

In [10]:
model = fit_dummy_regression(df)

In [11]:
df = add_residuals_to_df(df, model)

In [12]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 income   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     220.0
Date:                Sun, 25 Apr 2021   Prob (F-statistic):               0.00
Time:                        20:36:08   Log-Likelihood:                -26720.
No. Observations:               31665   AIC:                         5.356e+04
Df Residuals:                   31607   BIC:                         5.404e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              8.4198      0.327     25.758      0.000       7.779       9.061
C(year)[T.81]          0.0466      0.016      2.848      0.004       0.015       0.079
C(year)[T.82]          0.0994      0.016      6.100      0.000       0.067       0.131
C(year)[T.83]          0.1256      0.016      7.765      0.000       0.094       0.157
C(year)[T.84]          0.1887      0.016     11.685      0.000       0.157       0.220
C(year)[T.85]          0.2004      0.016     12.477      0.000       0.169       0.232
C(year)[T.86]          0.2415      0.016     15.118      0.000       0.210       0.273
C(year)[T.87]          0.2733      0.016     17.164      0.000       0.242       0.304
C(year)[T.88]          0.3443      0.016     21.641      0.000       0.313       0.375
C(year)[T.89]          0.4051      0.016     25.454      0.000       0.374       0.436
C(year)[T.90]          0.4276      0.016     26.931      0.000       0.397       0.459
C(year)[T.91]          0.4718      0.016     29.668      0.000       0.441       0.503
C(age)[T.26]           0.0876      0.021      4.087      0.000       0.046       0.130
C(age)[T.27]           0.1235      0.021      5.932      0.000       0.083       0.164
C(age)[T.28]           0.1694      0.021      8.173      0.000       0.129       0.210
C(age)[T.29]           0.2030      0.020      9.919      0.000       0.163       0.243
C(age)[T.30]           0.2220      0.020     10.872      0.000       0.182       0.262
C(age)[T.31]           0.2689      0.020     13.135      0.000       0.229       0.309
C(age)[T.32]           0.3006      0.021     14.610      0.000       0.260       0.341
C(age)[T.33]           0.3010      0.021     14.549      0.000       0.260       0.342
C(age)[T.34]           0.3073      0.021     14.765      0.000       0.267       0.348
C(age)[T.35]           0.3178      0.021     15.140      0.000       0.277       0.359
C(age)[T.36]           0.3421      0.021     15.977      0.000       0.300       0.384
C(age)[T.37]           0.3538      0.022     16.276      0.000       0.311       0.396
C(age)[T.38]           0.3554      0.022     15.864      0.000       0.311       0.399
C(age)[T.39]           0.3690      0.023     16.012      0.000       0.324       0.414
C(age)[T.40]           0.3791      0.024     15.954      0.000       0.333       0.426
C(age)[T.41]           0.4116      0.025     16.274      0.000       0.362       0.461
C(age)[T.42]           0.3637      0.026     14.180      0.000       0.313       0.414
C(age)[T.43]           0.3864      0.027     14.389      0.000       0.334       0.439
C(age)[T.44]           0.3783      0.028     13.323      0.000       0.323       0.434
C(age)[T.45]           0.3256      0.029     11.074      0.000       0.268       0.383
C(age)[T.46]           0.3797      0.0

In [13]:
df

income education work year  residuals
household age                                          
1         35   11.306651        17    1   91  -0.205778
          41   10.562056        11    0   83   0.671377
          41   10.603908        12    1   85  -0.147260
2         25   10.992065        12    1   88   0.508606
          29   10.400707        12    1   89  -0.346581
...                  ...       ...  ...  ...        ...
9317      26   10.300685        10    1   91  -0.096021
9345      35    9.834548        16    1   91  -1.506779
9347      31    9.397414        11    1   91  -1.197603
9348      27   10.190571        12    1   91  -0.543913
9354      38   10.833800        17    1   91  -0.716201

[31665 rows x 5 columns]